In [1]:
import math
from collections import Counter

# ENTROPY FUNCTION  
def entropy(data, target):
    values = [record[target] for record in data]
    counts = Counter(values)
    total = len(values)

    return sum((-count/total) * math.log(count/total, 2)
               for count in counts.values())


# INFORMATION GAIN  
def info_gain(data, attribute, target):
    total_entropy = entropy(data, target)

    # Unique values of attribute
    values = set(record[attribute] for record in data)
    total = len(data)

    subset_entropy = 0
    for value in values:
        subset = [record for record in data if record[attribute] == value]
        weight = len(subset) / total
        subset_entropy += weight * entropy(subset, target)

    return total_entropy - subset_entropy


# ID3 ALGORITHM  
def id3(data, attributes, target):
    values = [record[target] for record in data]

    # If all labels are same, return that label
    if len(set(values)) == 1:
        return values[0]

    # If no attributes left, return majority label
    if not attributes:
        return Counter(values).most_common(1)[0][0]

    # Select best attribute
    gains = {attr: info_gain(data, attr, target) for attr in attributes}
    best_attr = max(gains, key=gains.get)

    tree = {best_attr: {}}

    # Create branches
    values = set(record[best_attr] for record in data)
    for val in values:
        subset = [record for record in data if record[best_attr] == val]

        if not subset:
            tree[best_attr][val] = Counter(values).most_common(1)[0][0]
        else:
            new_attrs = [attr for attr in attributes if attr != best_attr]
            tree[best_attr][val] = id3(subset, new_attrs, target)

    return tree


# CLASSIFY NEW INSTANCE  
def classify(tree, record):
    attribute = next(iter(tree))  # root node
    value = record[attribute]
    next_node = tree[attribute][value]

    if isinstance(next_node, dict):
        return classify(next_node, record)
    else:
        return next_node


# RUNNING ID3  

dataset = [
    {'Outlook': 'Sunny', 'Temp': 'Hot', 'Humidity': 'High', 'Wind': 'Weak', 'Play': 'No'},
    {'Outlook': 'Sunny', 'Temp': 'Hot', 'Humidity': 'High', 'Wind': 'Strong', 'Play': 'No'},
    {'Outlook': 'Overcast', 'Temp': 'Hot', 'Humidity': 'High', 'Wind': 'Weak', 'Play': 'Yes'},
    {'Outlook': 'Rain', 'Temp': 'Mild', 'Humidity': 'High', 'Wind': 'Weak', 'Play': 'Yes'},
    {'Outlook': 'Rain', 'Temp': 'Cool', 'Humidity': 'Normal', 'Wind': 'Weak', 'Play': 'Yes'},
    {'Outlook': 'Rain', 'Temp': 'Cool', 'Humidity': 'Normal', 'Wind': 'Strong', 'Play': 'No'},
    {'Outlook': 'Overcast', 'Temp': 'Cool', 'Humidity': 'Normal', 'Wind': 'Strong', 'Play': 'Yes'},
    {'Outlook': 'Sunny', 'Temp': 'Mild', 'Humidity': 'High', 'Wind': 'Weak', 'Play': 'No'},
    {'Outlook': 'Sunny', 'Temp': 'Cool', 'Humidity': 'Normal', 'Wind': 'Weak', 'Play': 'Yes'},
    {'Outlook': 'Rain', 'Temp': 'Mild', 'Humidity': 'Normal', 'Wind': 'Weak', 'Play': 'Yes'},
]

attributes = ['Outlook', 'Temp', 'Humidity', 'Wind']
target = 'Play'

tree = id3(dataset, attributes, target)

print("Decision Tree (ID3)")
print(tree)

# TEST CLASSIFICATION  
test_sample = {'Outlook': 'Sunny', 'Temp': 'Cool', 'Humidity': 'High', 'Wind': 'Weak'}
result = classify(tree, test_sample)
print("\nTest Sample Classification:", result)


Decision Tree (ID3)
{'Outlook': {'Overcast': 'Yes', 'Rain': {'Wind': {'Weak': 'Yes', 'Strong': 'No'}}, 'Sunny': {'Temp': {'Hot': 'No', 'Mild': 'No', 'Cool': 'Yes'}}}}

Test Sample Classification: Yes
